In [1]:
### Import the required libraries
import numpy as np
import matplotlib.pyplot as plt 
import xarray as xr

import warnings

warnings.simplefilter("ignore")

import pandas as pd

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow import optimizers
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.layers import Input, Activation
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import layers
from sklearn.utils import shuffle
import tensorflow.keras.backend as kbackend
import tensorflow.keras.models
from keras.utils import np_utils
from keras.regularizers import l2

tf.compat.v1.disable_eager_execution()

import innvestigate
import innvestigate.utils as iutils

import os
import sys
path_list = os.path.abspath('').split('/')
path_src_XAIRT = ''
for link in path_list[:-1]:
    path_src_XAIRT = path_src_XAIRT+link+'/'
sys.path.append(path_src_XAIRT+'/src')

import XAIRT
import XAIRT.backend as backend
from XAIRT.backend.graph import getLayerIndexByName

2023-06-28 19:35:54.229534: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-28 19:35:54.442573: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/ohpc/pub/libs/gnu7/openmpi/netcdf/4.5.0/lib:/opt/ohpc/pub/libs/gnu7/openmpi/netcdf-fortran/4.4.4/lib:/opt/ohpc/pub/libs/gnu7/openmpi/hdf5/1.10.1/lib:/opt/ohpc/pub/mpi/openmpi-gnu7/1.10.7/lib:/opt/ohpc/pub/compiler/gcc/7.3.0/lib64:/home/shreyas/lis-1.4.43/installation/lib:/share/jdk-16.0.1/lib::
2023-06-28 19:35:54.442618: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ig

In [ ]:
def basal_topology_func(x):
    b = 1.0 - 0.1*x
    return b

def solution(nx, nt, L, T, M, basal_topology_func):

    if len(M) != nx + 1:
        raise ValueError('M specified but len(M) != nx + 1')
        
    dx = L/nx
    dt = T/nt
    x = np.linspace(0,L,nx+1)
    t = np.linspace(0,T,nt+1)

    b = basal_topology_func(x)

    A = 1e-16
    rho = 920.0
    g = 9.2 
    n = 3

    C = 2*A/(n+2) * (rho*g)**n * (1e3)**n

    h = np.zeros((nx+1,nt+1))
    H = np.zeros((nx+1,nt+1))
    h[:,0] = b
    h[0,:] = b[0]
    h[-1,:] = b[-1]

    H[:,0] = h[:,0] - b
    H[0,:] = h[0,:] - b[0]
    H[-1,:] = h[-1,:] - b[-1]

    for i in range(1,len(t)):

        D = C *((H[1:,i-1]+H[:nx,i-1])/2.0)**(n+2) * ((h[1:,i-1] - h[:nx,i-1])/dx)**(n-1)

        phi = -D*(h[1:,i-1]-h[:nx,i-1])/dx

        h[1:nx,i] = h[1:nx,i-1] + M[1:nx]*dt - dt/dx * (phi[1:]-phi[:nx-1])
        h[1:nx,i] = (h[1:nx,i] < b[1:nx]) * b[1:nx] + (h[1:nx,i] >= b[1:nx]) * h[1:nx,i]
        H[:,i] = np.maximum(h[:,i] - b, 0.)

        if not np.any(H[:,i]>=0.0):
            raise Exception("Something went wrong.")
            
    Volume = np.sum(H)*dx
    
    return H[int(nx/2),-1], h[int(nx/2),-1], Volume

In [ ]:
L = 30.
T = 10.
nx = 300
nt = 12000
samples = 10

M_samples = 0.01*np.random.rand(samples, nx+1)
H_samples = np.zeros((samples,1), dtype = np.float64)
Volume_samples = np.zeros((samples,1), dtype = np.float64)

for sample in range(samples):
    if (sample+1) % 10 == 0:
        print(f'Sample #{sample+1}', end='\r')
    H_samples[sample], _, Volume_samples[sample] = solution(nx, nt, L, T, M_samples[sample], basal_topology_func)

In [ ]:
keras.backend.clear_session()

inputs = Input(shape=(nx+1,))
dense1 = Dense(10, activation='relu')
dense2 = Dense(1, activation='linear', use_bias=False)

x = dense1(inputs)
x = dense2(x)

model = keras.Model(inputs=inputs, outputs=x) 

mod_h5 = 'model.h5'

checkpoint = ModelCheckpoint(mod_h5, monitor='val_loss',
                             verbose=1,save_best_only=True)
        
callbacks = [checkpoint]
model.compile(optimizer='adam',
            loss='mse', metrics=['mae'])

fit = model.fit(M_samples, H_samples,
            batch_size=10,
            epochs=10, 
            shuffle=True,
            validation_split = 0.2, 
            callbacks=callbacks)

In [ ]:
inputs = Input(shape=(nx+1,))
dense1 = Dense(10, activation='relu')
dense2 = Dense(1, activation='linear', use_bias=False)

x = dense1(inputs)
x = dense2(x)

best_model = keras.Model(inputs=inputs, outputs=x) 
best_model.load_weights(mod_h5)
best_model.compile(loss='mse', optimizer='adam',metrics=['mae'])

H_pred = best_model.predict(M_samples)

In [ ]:
analyzer_gradient = innvestigate.create_analyzer("lrp.alpha_1_beta_0", best_model)
a_alb0 = np.zeros((samples, nx+1), dtype = np.float64)
for sample in range(samples):
    a_alb0[sample] = analyzer_gradient.analyze(M_samples[sample][np.newaxis,:])
    a_alb0[sample] /= np.max(np.abs(a_alb0[sample]))

plt.plot(a_alb0[0])
plt.plot(a_alb0[10])

In [ ]:
analyzer_gradient = innvestigate.create_analyzer("lrp.z", best_model)
a_z = np.zeros((samples, nx+1), dtype = np.float64)
for sample in range(samples):
    a_z[sample] = analyzer_gradient.analyze(M_samples[sample][np.newaxis,:])
    a_z[sample] /= np.max(np.abs(a_z[sample]))

plt.plot(a_z[0])
plt.plot(a_z[10])

In [ ]:
def regression_letzgus(model, input_, y_ref, step_width=0.00005, max_it=10e4, method = "flood"):
    
    ### Finding a_ref for a given y_ref
    
    if method == "flood":
        
        model_part = Model(inputs=model.input,
                          outputs=model.layers[-2].output)
        a_ref = model_part.predict(input_)[0,:]
        a_ref = a_ref[:, np.newaxis]
        update = np.ones(a_ref.shape) * step_width
        
        y = model.predict(input_)
        
        counter = 0
        
        if y > y_ref:
            
            while y >= y_ref:
                
                a_ref = np.maximum(np.zeros(a_ref.shape),a_ref-update)
                y = np.dot(model.layers[-1].get_weights()[0][:,0], a_ref[:,0])
                counter +=1 
                print(f'iteration {counter} - y: {y}', end='\r')
                if counter > max_it:
                    print(f'! reference value {y_ref} was not reached within {round(max_it)} iterations!')
                    break
        else:
            
            while y <= y_ref:
                
                a_ref = np.maximum(np.zeros(a_ref.shape),a_ref+update)
                y = np.dot(model.layers[-1].get_weights()[0][:,0], a_ref[:,0])
                counter +=1 
                print(f'iteration {counter} - y: {y}', end='\r')
                if counter > max_it:
                    print(f'! reference value {y_ref} was not reached within {round(max_it)} iterations!')
                    break
                    
    else: 
        
        raise ValueError("The only methods available are : flood")
        
    return a_ref

def triplicated_model(best_model, a_ref):
    
    # get weights and biases
    W_in = best_model.layers[-2].get_weights()[0]
    W_out = best_model.layers[-1].get_weights()[0]
    bias_in = best_model.layers[-2].get_weights()[1]

    inputs = Input(shape=(nx+1,))

    # layer_dict = dict([(layer.name, layer) for layer in model.layers[:-2]])
    # weights = layer_dict['some_name'].get_weights()

    dense11 = Dense(10, activation='relu', name='dense11')
    dense12 = Dense(10, activation='relu', name='dense12')
    dense13 = Dense(10, activation='relu', name='dense13')
    dense21 = Dense(1, activation='linear', use_bias=False, name='dense21')
    dense22 = Dense(1, activation='linear', use_bias=False, name='dense22')
    dense23 = Dense(1, activation='linear', use_bias=False, name='dense23')

    x1 = dense11(inputs)
    x2 = dense12(inputs)
    x3 = dense13(inputs)

    x1 = dense21(x1)
    x2 = dense22(x2)
    x3 = dense23(x3)

    model = keras.Model(inputs=inputs, outputs=x1+x2+x3) 
    
    #     def getLayerIndexByName(model, layername):
    #         for idx, layer in enumerate(model.layers):
    #             if layer.name == layername:
    #                 return idx
    #         raise Exception(f"layername: {layername} not found.")
            
    model.layers[getLayerIndexByName(model, 'dense11')].set_weights([W_in, bias_in-a_ref[:,0]])
    model.layers[getLayerIndexByName(model, 'dense12')].set_weights([-W_in, -bias_in])                 
    model.layers[getLayerIndexByName(model, 'dense13')].set_weights([-W_in, -bias_in+a_ref[:,0]])
    
    model.layers[getLayerIndexByName(model, 'dense21')].set_weights([W_out])
    model.layers[getLayerIndexByName(model, 'dense22')].set_weights([W_out])
    model.layers[getLayerIndexByName(model, 'dense23')].set_weights([-W_out])
    
    model.compile(loss='mse', optimizer='adam',metrics=['mae'])
    
    return model

In [ ]:
index = 0
y = best_model.predict(M_samples[np.newaxis, index])
y_ref = y / 2.0

model_reg = keras.Model(inputs=inputs, outputs=x) 
model_reg.load_weights(mod_h5)
model_reg.compile(loss='mse', optimizer='adam',metrics=['mae'])
a_ref = regression_letzgus(model_reg, M_samples[np.newaxis, index], y_ref)

model_reg = keras.Model(inputs=inputs, outputs=x) 
model_reg.load_weights(mod_h5)
model_reg.compile(loss='mse', optimizer='adam',metrics=['mae'])
tri_model = triplicated_model(model_reg, a_ref)
y_reg = tri_model.predict(M_samples[np.newaxis, index])


y_ref, y-y_reg, y, y_reg

In [ ]:
index = 0
y = best_model.predict(M_samples[np.newaxis, index])
y_ref = y * 2.0

model_reg = keras.Model(inputs=inputs, outputs=x) 
model_reg.load_weights(mod_h5)
model_reg.compile(loss='mse', optimizer='adam',metrics=['mae'])
a_ref = regression_letzgus(model_reg, M_samples[np.newaxis, index], y_ref)

model_reg = keras.Model(inputs=inputs, outputs=x) 
model_reg.load_weights(mod_h5)
model_reg.compile(loss='mse', optimizer='adam',metrics=['mae'])
tri_model = triplicated_model(model_reg, a_ref)
y_reg = tri_model.predict(M_samples[np.newaxis, index])

y_ref, y-y_reg, y, y_reg

In [ ]:
def triplicated_model_LRP_compatible(best_model, a_ref):
    
    # get weights and biases
    W_in = best_model.layers[-2].get_weights()[0]
    W_out = best_model.layers[-1].get_weights()[0]
    bias_in = best_model.layers[-2].get_weights()[1]

    inputs = Input(shape=(nx+1,))

    # layer_dict = dict([(layer.name, layer) for layer in model.layers[:-2]])
    # weights = layer_dict['some_name'].get_weights()

    dense11 = Dense(10, activation='relu', name='dense11')
    dense12 = Dense(10, activation='relu', name='dense12')
    dense13 = Dense(10, activation='relu', name='dense13')
    dense21 = Dense(1, activation='linear', use_bias=False, name='dense21')
    dense22 = Dense(1, activation='linear', use_bias=False, name='dense22')
    dense23 = Dense(1, activation='linear', use_bias=False, name='dense23')

    x1 = dense11(inputs)
    x2 = dense12(inputs)
    x3 = dense13(inputs)

    x1 = dense21(x1)
    x2 = dense22(x2)
    x3 = dense23(x3)

    model1 = keras.Model(inputs=inputs, outputs=x1)
    model2 = keras.Model(inputs=inputs, outputs=x2) 
    model3 = keras.Model(inputs=inputs, outputs=x3) 
            
    model1.layers[getLayerIndexByName(model1, 'dense11')].set_weights([W_in, bias_in-a_ref[:,0]])
    model2.layers[getLayerIndexByName(model2, 'dense12')].set_weights([-W_in, -bias_in])                 
    model3.layers[getLayerIndexByName(model3, 'dense13')].set_weights([-W_in, -bias_in+a_ref[:,0]])
    
    model1.layers[getLayerIndexByName(model1, 'dense21')].set_weights([W_out])
    model2.layers[getLayerIndexByName(model2, 'dense22')].set_weights([W_out])
    model3.layers[getLayerIndexByName(model3, 'dense23')].set_weights([-W_out])
    
    model1.compile(loss='mse', optimizer='adam',metrics=['mae'])
    model2.compile(loss='mse', optimizer='adam',metrics=['mae'])
    model3.compile(loss='mse', optimizer='adam',metrics=['mae'])
    
    return model1, model2, model3

In [ ]:
index = 0
y = best_model.predict(M_samples[np.newaxis, index])
y_ref = y * 2.0

model_reg = keras.Model(inputs=inputs, outputs=x) 
model_reg.load_weights(mod_h5)
model_reg.compile(loss='mse', optimizer='adam',metrics=['mae'])
a_ref = regression_letzgus(model_reg, M_samples[np.newaxis, 0], y_ref)

model1, model2, model3 = triplicated_model_LRP_compatible(best_model, a_ref)
y1 = model1.predict(M_samples[np.newaxis, index])
y2 = model1.predict(M_samples[np.newaxis, index])
y3 = model1.predict(M_samples[np.newaxis, index])

analyzer_gradient1 = innvestigate.create_analyzer("lrp.alpha_1_beta_0", model1)
analyzer_gradient2 = innvestigate.create_analyzer("lrp.alpha_1_beta_0", model2)
analyzer_gradient3 = innvestigate.create_analyzer("lrp.alpha_1_beta_0", model3)
a1 = np.zeros((samples, nx+1), dtype = np.float64)
a2 = np.zeros((samples, nx+1), dtype = np.float64)
a3 = np.zeros((samples, nx+1), dtype = np.float64)
a  = np.zeros((samples, nx+1), dtype = np.float64)

for sample in range(samples):
    a1[sample] = analyzer_gradient1.analyze(M_samples[sample][np.newaxis,:])
    a2[sample] = analyzer_gradient2.analyze(M_samples[sample][np.newaxis,:])
    a3[sample] = analyzer_gradient3.analyze(M_samples[sample][np.newaxis,:])
    a[sample]  = a1[sample] + a2[sample] + a3[sample]
    a[sample]  = a[sample] / np.max(np.abs(a[sample]))
    
y, y_ref, y-(y1+y2+y3)

In [ ]:
#### Notice how sign of H_samples[index]-y_ref matters

index = 0
plt.plot(a[index])
H_samples[index]-y_ref

In [ ]:
#### Notice how sign of H_samples[index]-y_ref matters

index = 1
plt.plot(a[index])
H_samples[index]-y_ref

In [ ]:
#### Notice how sign of H_samples[index]-y_ref matters

index = 100
plt.plot(a[index])
H_samples[index]-y_ref

In [ ]:
index = 0
y = best_model.predict(M_samples[np.newaxis, index])
y_ref = y / 3.0

model_reg = keras.Model(inputs=inputs, outputs=x) 
model_reg.load_weights(mod_h5)
model_reg.compile(loss='mse', optimizer='adam',metrics=['mae'])
a_ref = regression_letzgus(model_reg, M_samples[np.newaxis, 0], y_ref)

model1, model2, model3 = triplicated_model_LRP_compatible(best_model, a_ref)
y1 = model1.predict(M_samples[np.newaxis, index])
y2 = model2.predict(M_samples[np.newaxis, index])
y3 = model3.predict(M_samples[np.newaxis, index])

analyzer_gradient1 = innvestigate.create_analyzer("lrp.alpha_1_beta_0", model1)
analyzer_gradient2 = innvestigate.create_analyzer("lrp.alpha_1_beta_0", model2)
analyzer_gradient3 = innvestigate.create_analyzer("lrp.alpha_1_beta_0", model3)
a1 = np.zeros((samples, nx+1), dtype = np.float64)
a2 = np.zeros((samples, nx+1), dtype = np.float64)
a3 = np.zeros((samples, nx+1), dtype = np.float64)
a  = np.zeros((samples, nx+1), dtype = np.float64)

for sample in range(samples):
    a1[sample] = analyzer_gradient1.analyze(M_samples[sample][np.newaxis,:])
    a2[sample] = analyzer_gradient2.analyze(M_samples[sample][np.newaxis,:])
    a3[sample] = analyzer_gradient3.analyze(M_samples[sample][np.newaxis,:])
    a[sample]  = a1[sample] + a2[sample] + a3[sample]
    a[sample]  = a[sample] / np.max(np.abs(a[sample]))
    
y, y_ref, y-(y1+y2+y3)

In [ ]:
index = 0
plt.plot(a[index])
H_samples[index]-y_ref

In [ ]:
index = 1
plt.plot(a[index])
H_samples[index]-y_ref

In [ ]:
index = 2
plt.plot(a[index])
H_samples[index]-y_ref